In [9]:
import torch
import torch.nn as nn

In [4]:
from transformers import BertModel, BertConfig
from transformers.models.bert.modeling_bert import *

In [6]:
config = BertConfig()

In [8]:
be = BertEncoder(config)

In [19]:
a = torch.randn(8, 10, 768)

In [20]:
b = be(a)

In [21]:
b[0].shape

torch.Size([8, 10, 768])

In [22]:
config.hidden_size

768